# running Region Based Analysis (RBA) - an hierarchical mixed effects models on the whole brain (aka a set of ROIs)

A notebook to run an RBA on fMRI data.  
OHBM Brainhack 2022 project  

Authors:  
Gang Chen [@afni-gangc](https://github.com/afni-gangc)  
Christopher Nolan [@crnolan](https://github.com/crnolan)    
Kelly Garner  [@kel-github](https://github.com/kel-github) 
Lea Waller  [@HippocampusGirl](https://github.com/HippocampusGirl)  
Daniel Tomasz [@danieltomasz](https://github.com/danieltomasz)  

Modelling task-based fMRI data often involves performing a GLM at each voxel and then correcting for many many many multiple comparisons.  

Here instead, we first summarise the data at the region of interest (ROI) level, and then perform a single hierarchical mixed effects model on all the ROIs at once.  

This provides advantages typical of Bayesian hiearchal modelling; information at upper levels of the hierarchy (e.g. across ROIs) can help inform estimates at lower levels (the estimate for each ROI) - aka shrinkage - and we avoid the multiple comparisons problem by instead providing the strength of evidenve for the effect of interest at each ROI.  

For a comprehensive introduction to this approach, see [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6635105/) and [this paper](https://apertureneuropub.cloud68.co/articles/46/index.html) by Gang Chen.

## Running the Notebook

We assume that you have followed the installation instructions detailed in the repo [ReadMe](https://github.com/crnolan/pyrba/blob/main/README.md) doc.  

**What we assume has happened before now**  

You have run an experiment manipulating or measuring a particular variable. For our example we are assuming we have collected theory of mind (ToM) measures and have fMRI data from participants performing a task.  

We want to know the association between ToM and activity in each of our ROIs. We perform a first-level analysis for each participant x voxel, and then for each ROI we average the effect of interest (e.g. a beta-coefficient or correlation score) across voxels.  

This produces a dataset containing the following fields:  

subject | ROI | y | x  

- x:  predictor variable (in this example a theory of mind score) (float)  
- y = DV of interest from first level analysis - e.g. mu beta coefficient from region of interest (ROI) (float)  
- ROI: the name of the region of interest (ROI) (string)  
- subject: ID (string) 

### load required modules

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr
import bambi as bmb
sns.set_theme(style='darkgrid')

### next we load the data  

We have a nice small set in the repo called 'data.txt' that you can try this on  

In [ ]:
df = pd.read_csv('data.txt', delimiter = '\s')
df.head(5)

Next we define the model. Luckily for us, Tal Yarkoni & Jake Westfall developed this awesome [Bambi package](https://osf.io/rv7sn) which allows us to specify our linear mixed effects model using the notation we know so well in the neurosciences and social sciences more broadly (If you don't know the notation then see [this excellent introduction](https://bodo-winter.net/tutorial/bw_LME_tutorial2.pdf) from Bodo Winter). The package takes our formula and specifies our model in [PyMC](https://www.pymc.io/welcome.html) parlance, ready for passing to PyMC.  

We define a model where our data (_y_) is predicted by our ToM measure (_x_), an intercept for each subject (_(1|subject)_), and a ToM x ROI interaction (_(x|ROI)_).

In [ ]:
model = bmb.Model("y ~ x + (1|subject) + (x|ROI)", data=df)

We then fit the model using PyMC. We are going to use the default priors (see the model graph below), and the No U-Turn Sampler ([NUTS](https://arxiv.org/abs/1111.4246)). 



In [ ]:
fitted = model.fit(tune=4000, 
                   draws=1000, 
                   chains=16, 
                   method='nuts_numpyro',
                   nuts_kwargs=dict(max_tree_depth=100))

A nice feature of PyMC is that we can easily print the model graph, which shows us the assumptions we've made about the parameters in our model.  

In [ ]:
model.graph()

As you can see above, we assume that the random effects from the model come from normal distributions whose variance is drawn from a half normal (note this deviates from the original implementation in R, but we are working on this). Next we plot the psoterior estimates, and the chains to check that the model converged.  

We also print out a summary of the model. We want to check the $\hat{R}$ is around 1.

In [ ]:
az.plot_trace(fitted, figsize=(20, 35))
az.summary(fitted)

We now want to check the results look somewhat like the output from Gang's original R implementation, so what follows is some data-wrangling and then a plot comparison...

In [ ]:
rois = fitted['posterior']['x|ROI'].stack(y=['draw', 'chain']).to_pandas().transpose()
rois.columns.name = 'ROI'
rois = rois.stack()
rois.name = 'value'
rois

In [ ]:
roi_means = rois.groupby(['ROI']).mean().sort_values(ascending=False)

In [ ]:
roi_df = rois.to_frame()
roi_df['roi_mean'] = roi_df.groupby(['ROI']).transform('mean')
roi_df